# COMP 561 Final Project

## Objectives
1. Identifying a set of bound and non‐bound DNA sequences for a given TF based on existing experimental data.
2. Calculating the DNA physical properties of each sequence.
3. Training a machine learning classifier to distinguish between bound and unbound sites.

## 1. Identifiying bound/non-bound sequences

In [14]:
# input file: set of genomic regions that are active regulatory regions in GM12878
# a regulatory region is characterized by: chromosome #, start nt, end nt, ID (ex. chr2.3 is the 3rd site on chr2)
reg_regions = []
with open('data/wgEncodeRegTfbsClusteredV3.GM12878.merged.bed', 'r') as file:
    for line in file:
        reg_regions.append(line.rstrip().split('\t'))

In [15]:
reg_regions[0]

['chr1', '237550', '237989', 'chr1.1']

In [22]:
# input file: set of genomic coordinates of transcription factor binding sites for several transcription factors
# binding_sites is all bound sequences
binding_sites = []
with open ('data/factorbookMotifPos.txt', 'r') as file:
    for line in file:
        binding_sites.append(line.rstrip().split('\t')[1:])

In [23]:
binding_sites[0]

['chr1', '10461', '10476', 'UA1', '2.02', '-']

In [ ]:
# difference between reg_regions and binding_sites is the set of non_binding_sites